## Fully automated degradation process

1. Generate degraded MOF structure (select degradation percent)
2. Solvate degraded MOF with water and drug molecules (and maybe deleted linkers)
3. Assign force field parameters to solvated structure
4. Generate LAMMPS files
5. Run simulation

## Questions

- Is Node + Linker degradation fine?
- Which linkers/nodes to delete for degradation (start from outer shell??)
- Should we add the degraded linkers/nodes in solution??

In [1]:
import os
from thermof.trajectory.tools import center_of_mass
from file_io import write_xyz, read_xyz, write_cif
from degradation_tools import pack_mof, degrade_mof, MW
from irmof1 import irmof1
from packmol import Packmol

In [2]:
W_WATER = 0.1       # Water weight fraction
W_PZA = 0.5         # PZA weight fraction
W_NIZ = 0.0         # NIZ weight fraction
W_RFP = 0.0         # RFP weight fraction
D_MOF = (0.5, 0.5)  # Degradation fractions (linkers, nodes)
P_MOF = [2, 2, 2]   # MOF packing coefficient

## Generating MOF

In [3]:
mof = irmof1.copy()
packed_mof = pack_mof(mof['atoms'], mof['coordinates'], mof['cell'], P_MOF)
dmof_file = 'IRMOF1-%s-L%i-N%i.xyz' % (''.join([str(i) for i in P_MOF]), D_MOF[0] * 100, D_MOF[1] * 100)
degrade_mof(packed_mof, D_MOF, os.path.join('packmol', dmof_file))

96 / 192 linkers will be deleted...
32 / 64 nodes will be deleted...
Packing linkers and nodes...
Deleting linker and note atoms...
Writing to file...
Done! Saved as -> packmol/IRMOF1-222-L50-N50.xyz


## Solvating with Packmol

In [4]:
# structures = {'frac'}

In [5]:
edge_tol = 2                  # Empty space on the edges
solv_shell = [20, 20, 20]     # Solvation shell length
box_size = [int(packed_mof['cell'][i]) + 2 * solv_shell[i] for i in range(3)] 
pmol = Packmol()

dmof_position = {'fixed': '%.1f %.1f %.1f 0. 0. 0.' % (box_size[0] / 2, box_size[1] / 2, box_size[2] / 2), 'centerofmass': ''}
pmol.add_structure({'structure': dmof_file, 'number': '1', 'position': dmof_position})

### WATER ----------------------------
n_water = int((MW['IRMOF1'] + MW['H2O']) * W_WATER / MW['H2O'])
water_position = {'inside box': '%.1f %.1f %.1f %.1f %.1f %.1f' % (edge_tol, edge_tol, edge_tol, box_size[0] - edge_tol, box_size[1] - edge_tol, box_size[2] - edge_tol)}
print('Adding %i water molecules to ' % n_water, water_position)
pmol.add_structure({'structure': 'water.xyz', 'number': n_water, 'position': water_position})

### PZA -------------------------------
n_pza = int((MW['IRMOF1'] + MW['PZA']) * W_PZA / MW['PZA'])
pza_pad = int(sum(D_MOF) / len(D_MOF) * packed_mof['cell'][0] / 2 * (0.5))
print('PZA padding: %.1f' % pza_pad)
pza_pos = [solv_shell[0] + pza_pad, solv_shell[1] + pza_pad, solv_shell[2] + pza_pad] 
pza_pos += [box_size[0] - solv_shell[0] - pza_pad, box_size[1] - solv_shell[1] - pza_pad, box_size[2] - solv_shell[2] - pza_pad]
pza_position = {'inside box': '%s' % ' '.join([str(round(i, 1)) for i in pza_pos])}
print('Adding %i PZA molecules to ' % n_pza, pza_position)
pmol.add_structure({'structure': 'pza.xyz', 'number': n_pza, 'position': pza_position})

### LINKER ----------------------------
n_linker = int(D_MOF[0] * MW['IRMOF1'] / MW['LINKER'])
linker_position = water_position
print('Adding %i linker molecules to ' % n_linker, linker_position)
pmol.add_structure({'structure': 'linker.xyz', 'number': n_linker, 'position': linker_position})

### NODE ------------------------------
n_node = int(D_MOF[1] * MW['IRMOF1'] / MW['NODE'])
node_position = water_position
print('Adding %i node molecules to ' % n_node, node_position)
pmol.add_structure({'structure': 'node.xyz', 'number': n_node, 'position': node_position})

run_dir = 'pmol-temp'
source_dir = 'packmol'

Adding 34 water molecules to  {'inside box': '2.0 2.0 2.0 89.0 89.0 89.0'}
PZA padding: 6.0
Adding 25 PZA molecules to  {'inside box': '26 26 26 65 65 65'}
Adding 18 linker molecules to  {'inside box': '2.0 2.0 2.0 89.0 89.0 89.0'}
Adding 11 node molecules to  {'inside box': '2.0 2.0 2.0 89.0 89.0 89.0'}


In [6]:
pmol.run(run_dir, source_dir)

## Assign force field parameters and initialize LAMMPS

In [7]:
packed_xyz = read_xyz(os.path.join(run_dir, 'packed.xyz'))[0]
cif_file = os.path.join(run_dir, 'packed.cif')
write_cif(cif_file, packed_xyz['atoms'], packed_xyz['coordinates'], cell=box_size + [90, 90, 90], fractional=True)

In [8]:
from thermof import Simulation
from thermof import Parameters

In [9]:
lammps_dir = 'lammps-temp'
simpar = Parameters()
sim = Simulation(mof=os.path.abspath(cif_file), parameters=simpar)
sim.set_dir(lammps_dir)
simpar.thermof['fix'] = ['MIN', 'NPT', 'NVT']
simpar.thermof['min']['edif'] = 1e-3
simpar.thermof['npt']['steps'] = 1000000
simpar.thermof['nvt']['steps'] = 1000000
simpar.thermof['thermo_style'] = ['step', 'temp', 'press', 'pe', 'etotal', 'emol', 'epair', 'vol', 'lx', 'ly', 'lz']
simpar.job['nodes'] = 4
simpar.job['ppn'] = 28
simpar.job['walltime'] = '36:00:00'
simpar.job['name'] = 'P%.1f-W%.1f-DL%.1f-DN%.1f' % (W_PZA, W_WATER, D_MOF[0], D_MOF[1])

In [10]:
sim.initialize()

Removing existing simulation directory -> lammps-temp
I. Writing Lammps input and data files...
No bonds reported in cif file - computing bonding..
Molecules found in the framework, separating.


Files created! -> lammps-temp
II. Updating Lammps input file -> lammps-temp/in.packed
Adding fixes: MIN | NPT | NVT
Updating simulation parameters...
III. Writing slurm job submission file -> lammps-temp/job.P0.5-W0.1-DL0.5-DN0.5
Simulation parameters saved -> lammps-temp/simpar.yaml
Done!


## LAMMPS

In [ ]:
# Modify input files
# Create job submission file

In [ ]:
linker_atoms = [irmof1['atoms'][i] for i in irmof1['linkers'][0]]
linker_coors = [irmof1['coordinates'][i] for i in irmof1['linkers'][0]]

node_atoms = [irmof1['atoms'][i] for i in irmof1['nodes'][0]]
node_coors = [irmof1['coordinates'][i] for i in irmof1['nodes'][0]]
write_xyz('node.xyz', node_atoms, node_coors)
write_xyz('linker.xyz', linker_atoms, linker_coors)